In [4]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import random
from func import loaddata,evaluate,preprocess,get_answer,MAP
import os

In [5]:
num_user,num_item,data=loaddata(path="data/train.csv")
#pos_train,pos_val,neg_val=preprocess(num_user,num_item,data)   
#np.save("data/split.npy",[pos_train,pos_val,neg_val])
pos_train,pos_val,neg_val,val=np.load("data/split.npy",allow_pickle=True)


In [6]:
def writefile(dump,user,item,num_user,pos_train,pos_val):
    answer=get_answer(user,item,num_user,pos_train.union(pos_val))
    file=open(dump,"w")
    file.write("UserId,ItemId\n")
    for i in range(num_user):
        file.write("%d,%s\n"%(i," ".join(answer[i])))
    file.close()


In [19]:
sig=nn.Sigmoid()
pred=torch.zeros([num_user,num_item])
#best_net=["model/32_0.432.pkl","model/16_0.400.pkl"]
for net in os.listdir("select"):
    user,item=torch.load("select/"+net)
    pred+=sig(torch.matmul(user,item.T))
    #answer=get_answer(user,item,num_user,pos_train)
    #print(MAP(answer,pos_val,num_user,val))

In [22]:
exclude=pos_train.union(pos_val)
sort, indices = torch.sort(pred, dim=1,descending=True)
del sort
indices=indices.numpy()
answer=np.empty([num_user,50])
for i in range(num_user):
    ite=0
    for j,ele in enumerate(indices[i]):
        if ite>=50:
            break
        elif (i,ele) not in exclude:
            answer[i][ite]=ele
            ite+=1
answer=answer.astype("int")

In [21]:
print(MAP(answer,pos_val,num_user,val))

0.14833424298661638


In [23]:
answer=answer.astype("int").astype("str")
dump="output/ensemble.csv"
file=open(dump,"w")
file.write("UserId,ItemId\n")
for i in range(num_user):
    file.write("%d,%s\n"%(i," ".join(answer[i])))
file.close()

In [27]:
(1,2) in set([(1,2),(3,4)])

True